What is BERT?
BERT is an open-source library created in 2018 at Google. It's a new technique for NLP and it takes a completely different approach to training models than any other technique.

BERT is an acronym for Bidirectional Encoder Representations from Transformers. That means unlike most techniques that analyze sentences from left-to-right or right-to-left, BERT goes both directions using the Transformer encoder. Its goal is to generate a language model.

This gives it incredible accuracy and performance on smaller data sets which solves a huge problem in natural language processing.

While there is a huge amount of text-based data available, very little of it has been labeled to use for training a machine learning model. Since most of the approaches to NLP problems take advantage of deep learning, you need large amounts of data to train with.

You really see the huge improvements in a model when it has been trained with millions of data points. To help get around this problem of not having enough labelled data, researchers came up with ways to train general purpose language representation models through pre-training using text from around the internet.

These pre-trained representation models can then be fine-tuned to work on specific data sets that are smaller than those commonly used in deep learning. These smaller data sets can be for problems like sentiment analysis or spam detection. This is the way most NLP problems are approached because it gives more accurate results than starting with the smaller data set.

That's why BERT is such a big discovery. It provides a way to more accurately pre-train your models with less data. The bidirectional approach it uses means it gets more of the context for a word than if it were just training in one direction. With this additional context, it is able to take advantage of another technique called masked LM.

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
df = pd.read_csv('../input/nlp-getting-started/train.csv')
df_test = pd.read_csv('../input/nlp-getting-started/test.csv') 

Clean the data : 
BERT can handle punctuation, smileys etc. Of course, smileys contribute a lot to sentiment analysis. So, don't remove them. Next, it would be fair to replace @mentions and links with some special tokens, because the model will probably never see them again in the future.
It is advisable to  fine-tune BERT with additional corpus, and after fine-tune with Twitter corpus. Or do it simultaneously. More training samples is generally better.


In [3]:
"""df['text'] = df['text'].str.lower() #lowercase
df_test['text'] = df_test['text']
df['text'] = df['text'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
df_test['text'] = df_test['text'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
# remove numbers
#remove.............. (#re sub / search/ ..)
df['text'] = df['text'].apply(lambda elem: re.sub(r"\d+", "", elem))
df_test['text'] = df_test['text'].apply(lambda elem: re.sub(r"\d+", "", elem))
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)



df['text'] = df['text'].apply(lambda x: remove_URL(x))
df_test['text'] = df_test['text'].apply(lambda x: remove_URL(x))
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

df['text'] = df['text'].apply(lambda x: remove_html(x))
df_test['text'] = df_test['text'].apply(lambda x: remove_html(x))
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

import string
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

df['text'] = df['text'].apply(lambda x: remove_punct(x))
df_test['text'] = df_test['text'].apply(lambda x: remove_punct(x)) """ 

'df[\'text\'] = df[\'text\'].str.lower() #lowercase\ndf_test[\'text\'] = df_test[\'text\']\ndf[\'text\'] = df[\'text\'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\\w+:\\/\\/\\S+)|^rt|http.+?", "", elem))  \ndf_test[\'text\'] = df_test[\'text\'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\\w+:\\/\\/\\S+)|^rt|http.+?", "", elem))  \n# remove numbers\n#remove.............. (#re sub / search/ ..)\ndf[\'text\'] = df[\'text\'].apply(lambda elem: re.sub(r"\\d+", "", elem))\ndf_test[\'text\'] = df_test[\'text\'].apply(lambda elem: re.sub(r"\\d+", "", elem))\ndef remove_URL(text):\n    url = re.compile(r\'https?://\\S+|www\\.\\S+\')\n    return url.sub(r\'\',text)\n\n\n\ndf[\'text\'] = df[\'text\'].apply(lambda x: remove_URL(x))\ndf_test[\'text\'] = df_test[\'text\'].apply(lambda x: remove_URL(x))\ndef remove_html(text):\n    html=re.compile(r\'<.*?>\')\n    return html.sub(r\'\',text)\n\ndf[\'text\'] = df[\'text\'].apply(lambda x: remove_html(x))\ndf_te

In [4]:
"""def text_to_wordlist(text, remove_stop_words=False, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    return(text) 

df['text'] = df['text'].apply(lambda x: text_to_wordlist(x))
df_test['text'] = df_test['text'].apply(lambda x: text_to_wordlist(x)) """ 

'def text_to_wordlist(text, remove_stop_words=False, stem_words=False):\n    # Clean the text, with the option to remove stop_words and to stem words.\n\n    # Clean the text\n    text = re.sub(r"[^A-Za-z0-9]", " ", text)\n    text = re.sub(r"what\'s", "", text)\n    text = re.sub(r"What\'s", "", text)\n    text = re.sub(r"\'s", " ", text)\n    text = re.sub(r"\'ve", " have ", text)\n    text = re.sub(r"can\'t", "cannot ", text)\n    text = re.sub(r"n\'t", " not ", text)\n    text = re.sub(r"I\'m", "I am", text)\n    text = re.sub(r" m ", " am ", text)\n    text = re.sub(r"\'re", " are ", text)\n    text = re.sub(r"\'d", " would ", text)\n    text = re.sub(r"\'ll", " will ", text)\n    text = re.sub(r"60k", " 60000 ", text)\n    text = re.sub(r" e g ", " eg ", text)\n    text = re.sub(r" b g ", " bg ", text)\n    text = re.sub(r"\x00s", "0", text)\n    text = re.sub(r" 9 11 ", "911", text)\n    text = re.sub(r"e-mail", "email", text)\n    text = re.sub(r"\\s{2,}", " ", text)\n    text 

In [5]:
"""# replace strange punctuations and raplace diacritics
from unicodedata import category, name, normalize

def remove_diacritics(s):
    return ''.join(c for c in normalize('NFKD', s.replace('ø', 'o').replace('Ø', 'O').replace('⁻', '-').replace('₋', '-'))
                  if category(c) != 'Mn')

special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}
def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
  
    text = remove_diacritics(text)
    return text

df['text'] = df['text'].apply(lambda x: remove_diacritics(x))
df['text'] = df['text'].apply(lambda x: clean_special_punctuations(x))
df_test['text'] = df_test['text'].apply(lambda x: clean_special_punctuations(x))
df_test['text'] = df_test['text'].apply(lambda x: remove_diacritics(x)) """

'# replace strange punctuations and raplace diacritics\nfrom unicodedata import category, name, normalize\n\ndef remove_diacritics(s):\n    return \'\'.join(c for c in normalize(\'NFKD\', s.replace(\'ø\', \'o\').replace(\'Ø\', \'O\').replace(\'⁻\', \'-\').replace(\'₋\', \'-\'))\n                  if category(c) != \'Mn\')\n\nspecial_punc_mappings = {"—": "-", "–": "-", "_": "-", \'”\': \'"\', "″": \'"\', \'“\': \'"\', \'•\': \'.\', \'−\': \'-\',\n                         "’": "\'", "‘": "\'", "´": "\'", "`": "\'", \'\u200b\': \' \', \'\xa0\': \' \',\'،\':\'\',\'„\':\'\',\n                         \'…\': \' ... \', \'\ufeff\': \'\'}\ndef clean_special_punctuations(text):\n    for punc in special_punc_mappings:\n        if punc in text:\n            text = text.replace(punc, special_punc_mappings[punc])\n  \n    text = remove_diacritics(text)\n    return text\n\ndf[\'text\'] = df[\'text\'].apply(lambda x: remove_diacritics(x))\ndf[\'text\'] = df[\'text\'].apply(lambda x: clean_special_pu

In [6]:
""""# clean numbers
def clean_number(text):
   
    text = re.sub(r'(\d+)([a-zA-Z])', '\g<1> \g<2>', text)
    text = re.sub(r'(\d+) (th|st|nd|rd) ', '\g<1>\g<2> ', text)
    text = re.sub(r'(\d+),(\d+)', '\g<1>\g<2>', text)
    
#     text = re.sub('[0-9]{5,}', '#####', text)
#     text = re.sub('[0-9]{4}', '####', text)
#     text = re.sub('[0-9]{3}', '###', text)
#     text = re.sub('[0-9]{2}', '##', text)
    
    return text

df['text'] = df['text'].apply(lambda x: clean_number(x))
df_test['text'] = df_test['text'].apply(lambda x: clean_number(x))"""

'"# clean numbers\ndef clean_number(text):\n   \n    text = re.sub(r\'(\\d+)([a-zA-Z])\', \'\\g<1> \\g<2>\', text)\n    text = re.sub(r\'(\\d+) (th|st|nd|rd) \', \'\\g<1>\\g<2> \', text)\n    text = re.sub(r\'(\\d+),(\\d+)\', \'\\g<1>\\g<2>\', text)\n    \n#     text = re.sub(\'[0-9]{5,}\', \'#####\', text)\n#     text = re.sub(\'[0-9]{4}\', \'####\', text)\n#     text = re.sub(\'[0-9]{3}\', \'###\', text)\n#     text = re.sub(\'[0-9]{2}\', \'##\', text)\n    \n    return text\n\ndf[\'text\'] = df[\'text\'].apply(lambda x: clean_number(x))\ndf_test[\'text\'] = df_test[\'text\'].apply(lambda x: clean_number(x))'

In [7]:
# de-contract the contraction
def decontracted(text):
    # specific
    text = re.sub(r"(W|w)on(\'|\’)t ", "will not ", text)
    text = re.sub(r"(C|c)an(\'|\’)t ", "can not ", text)
    text = re.sub(r"(Y|y)(\'|\’)all ", "you all ", text)
    text = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", text)

    # general
    text = re.sub(r"(I|i)(\'|\’)m ", "i am ", text)
    text = re.sub(r"(A|a)in(\'|\’)t ", "is not ", text)
    text = re.sub(r"n(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)re ", " are ", text)
    text = re.sub(r"(\'|\’)s ", " is ", text)
    text = re.sub(r"(\'|\’)d ", " would ", text)
    text = re.sub(r"(\'|\’)ll ", " will ", text)
    text = re.sub(r"(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)ve ", " have ", text)
    return text

df['text'] = df['text'].apply(lambda x: decontracted(x))
df_test['text'] = df_test['text'].apply(lambda x: decontracted(x))

In [8]:
import string
regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']
all_punct = list(set(regular_punct + extra_punct))
# do not spacing - and .
all_punct.remove('-')
all_punct.remove('.')

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

df['text'] = df['text'].apply(lambda x: spacing_punctuation(x))
df_test['text'] = df_test['text'].apply(lambda x: spacing_punctuation(x))

In [9]:
mis_connect_list = ['(W|w)hat', '(W|w)hy', '(H|h)ow', '(W|w)hich', '(W|w)here', '(W|w)ill']
mis_connect_re = re.compile('(%s)' % '|'.join(mis_connect_list))

mis_spell_mapping = {'whattsup': 'WhatsApp', 'whatasapp':'WhatsApp', 'whatsupp':'WhatsApp', 
                      'whatcus':'what cause', 'arewhatsapp': 'are WhatsApp', 'Hwhat':'what',
                      'Whwhat': 'What', 'whatshapp':'WhatsApp', 'howhat':'how that',
                      # why
                      'Whybis':'Why is', 'laowhy86':'Foreigners who do not respect China',
                      'Whyco-education':'Why co-education',
                      # How
                      "Howddo":"How do", 'Howeber':'However', 'Showh':'Show',
                      "Willowmagic":'Willow magic', 'WillsEye':'Will Eye', 'Williby':'will by'}
def spacing_some_connect_words(text):
    """
    'Whyare' -> 'Why are'
    """
    ori = text
    for error in mis_spell_mapping:
        if error in text:
            text = text.replace(error, mis_spell_mapping[error])
            
    # what
    text = re.sub(r" (W|w)hat+(s)*[A|a]*(p)+ ", " WhatsApp ", text)
    text = re.sub(r" (W|w)hat\S ", " What ", text)
    text = re.sub(r" \S(W|w)hat ", " What ", text)
    # why
    text = re.sub(r" (W|w)hy\S ", " Why ", text)
    text = re.sub(r" \S(W|w)hy ", " Why ", text)
    # How
    text = re.sub(r" (H|h)ow\S ", " How ", text)
    text = re.sub(r" \S(H|h)ow ", " How ", text)
    # which
    text = re.sub(r" (W|w)hich\S ", " Which ", text)
    text = re.sub(r" \S(W|w)hich ", " Which ", text)
    # where
    text = re.sub(r" (W|w)here\S ", " Where ", text)
    text = re.sub(r" \S(W|w)here ", " Where ", text)
    # 
    text = mis_connect_re.sub(r" \1 ", text)
    text = text.replace("What sApp", 'WhatsApp')
    
    
    return text

df['text'] = df['text'].apply(lambda x: spacing_some_connect_words(x))
df_test['text'] = df_test['text'].apply(lambda x: spacing_some_connect_words(x))

In [10]:
#https://www.kaggle.com/sunnymarkliu/more-text-cleaning-to-increase-word-coverage

In [11]:
df.head() 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13 , 000 people receive # wildfires evacuatio...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1


In [12]:
#Importing pre-trained DistilBERT model and tokenizer
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [13]:
df = df[:4000]

we need to tokenize our texts. BERT was trained using the WordPiece tokenization. It means that a word can be broken down into more than one sub-words. For example, if I tokenize the sentence “Hi, my name is Dima” I’ll get:
tokenizer.tokenize('Hi my name is Dima')
# OUTPUT
['hi', 'my', 'name', 'is', 'dim', '##a']

This kind of tokenization is beneficial when dealing with out of vocabulary words, and it may help better represent complicated words. The sub-words are constructed during the training time and depend on the corpus the model was trained on. We could use any other tokenization technique of course, but we’ll get the best results if we tokenize with the same tokenizer the BERT model was trained on. 

In [14]:
#we’ll tokenize and process all sentences together as a batch 
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

The code below creates the tokenizer, tokenizes each review, adds the special [CLS] token, and then takes only the first 512 tokens for both train and test sets.


Next, we need to convert each token in each review to an id as present in the tokenizer vocabulary. If there’s a token that is not present in the vocabulary, the tokenizer will use the special [UNK] token and use its id.

Finally, we need to pad our input so it will have the same size of 512. It means that for any review that is shorter than 512 tokens, we’ll add zeros to reach 512 tokens.

Our target variable is currently a list of 1 and 0 strings. We’ll convert it to numpy arrays of booleans.

In [15]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [16]:
np.array(padded).shape

(4000, 82)

In [17]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(4000, 82)

In [18]:
#The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [19]:
features = last_hidden_states[0][:,0,:].numpy()
features[0].shape



(768,)

In [20]:
labels = df['target']

In [21]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [22]:
import sklearn
from sklearn.model_selection import GridSearchCV
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)
print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


best parameters:  {'C': 5.263252631578947}
best scrores:  0.7803333333333333


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
lr_clf = LogisticRegression(C = 31.579015789473683)


In [24]:
lr_clf.fit(train_features, train_labels)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=31.579015789473683)

In [25]:
lr_clf.score(test_features, test_labels)

0.753

In [26]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.519 (+/- 0.03)


/opt/conda/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [27]:
tokenized_t = df_test['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [28]:
max_len = 0
for i in tokenized_t.values:
    if len(i) > max_len:
        max_len = len(i)

In [29]:
padded_t = np.array([i + [0]*(max_len-len(i)) for i in tokenized_t.values])
np.array(padded_t).shape

(3263, 73)

In [30]:
attention_mask_t = np.where(padded_t != 0, 1, 0)

In [31]:
input_ids = torch.tensor(padded_t)  
input_ids

tensor([[  101,  2074,  3047,  ...,     0,     0,     0],
        [  101,  2657,  2055,  ...,     0,     0,     0],
        [  101,  2045,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  2665,  2240,  ...,     0,     0,     0],
        [  101, 12669,  3314,  ...,     0,     0,     0],
        [  101,  1001,  2103,  ...,     0,     0,     0]])

In [32]:
attention_mask_t = torch.tensor(attention_mask_t)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask_t)

In [33]:
val_features = last_hidden_states[0][:,0,:].numpy() 

In [34]:
y_pred = lr_clf.predict(val_features)
y_pred

array([0, 1, 1, ..., 1, 0, 1])

In [35]:
# Create submission file
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)